In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import timeit as timeit
import AMM_sim_functions as sim
import fast_AMM_sim_functions as fsim

In [2]:
# Market Parameters

T = 1 # Time horizon
N = 1440 # Number of periods
dt = T/N # Time increment
buy = 2500.*dt # Constant trade size for systematic buyers
sell = -2500.*dt # Constant trade size for systematic sellers
eta0 = 0.005 # CEX proportional cost
eta1 = 0.0025 # CPMM proportional cost
X = 30000000 # Initial CPMM Dollar Reserves
Y = 10000 # Initial CPMM Asset Reserves
S = X/Y # Initial CEX Price
sigma = 0.05 # Standard deviation of CEX price shock
mu = 0  # Mean of CEX price shock
M = 10000 # Total number of sims
time = np.array([i * dt for i in range(N+1)]) # Time grid

In [3]:
S0 = sim.CEX_Price(S, mu, sigma, dt, N, M) # Generate price series

filtr_bfs , filtr_sfs =  sim.generate_trade_filters(N, M) # Generate buyer-first/seller-first trade filter

In [4]:
%%time

# Original Simulation
v = sim.simulation_summary(M, N, T, dt, buy, sell, eta0, eta1, S0, X, Y, filtr_bfs, filtr_sfs)
v

CPU times: user 1.64 s, sys: 227 ms, total: 1.86 s
Wall time: 1.87 s


array([1.42261448e+04, 1.42501029e+04, 1.03536181e+03, 5.99774287e+07,
       5.99960844e+07])

In [5]:
%%time

# Fast Simulation
vf = fsim.fast_simulation_summary(M, N, T, dt, buy, sell, eta0, eta1, S0, X, Y, filtr_bfs, filtr_sfs)
vf

CPU times: user 700 ms, sys: 117 ms, total: 817 ms
Wall time: 888 ms


array([1.42261448e+04, 1.42501029e+04, 1.03536181e+03, 5.99774287e+07,
       5.99960844e+07])

In [6]:
# Compare values to ensure they are the same (up to numerical error)
print(vf-v)
print(np.max(np.abs(vf-v)))

[ 3.41242412e-09  1.50430424e-09 -9.77706804e-12  3.72529030e-08
  0.00000000e+00]
3.725290298461914e-08


In [7]:
# Measure execution time of original code
execution_time = timeit.timeit(lambda: sim.simulation_summary(M, N, T, dt, buy, sell, eta0, eta1, S0, X, Y, filtr_bfs, filtr_sfs), number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")

Average execution time: 1.907697 seconds


In [8]:
# Measure execution time of fast code
execution_time = timeit.timeit(lambda: fsim.fast_simulation_summary(M, N, T, dt, buy, sell, eta0, eta1, S0, X, Y, filtr_bfs, filtr_sfs),number=100)

print(f"Average execution time: {execution_time / 100:.6f} seconds")

Average execution time: 0.398382 seconds
